# CPMC


Consider the Hamiltonian

\begin{equation*}
H = H_1 + H_2 = -t\left(c_{i\sigma}^{\dagger}c_{j\sigma} + \text{h.c.}\right) + U_{ij}n_{i\sigma}n_{j\sigma'}.
\end{equation*}

Trotter decompose the projector as
\begin{equation*}
\exp(-\Delta\tau H) = \exp(-\Delta\tau H_1/2)\exp(-\Delta\tau H_2)\exp(-\Delta\tau H_1/2) + O(\Delta\tau^3).
\end{equation*}

Using the Hubbard-Stratonovich transformation, the interaction term can be written as

\begin{equation*}
\begin{split}
\exp(-\Delta\tau U_{ij}n_{i\sigma}n_{j\sigma'}) & = \Pi_{ij}\left[ \exp \left(-\frac{\Delta\tau U_{ij}}{2}(n_{i\sigma}+n_{j\sigma'})\right) \sum_{x_{ij}^{\sigma\sigma'}=\pm 1} \frac{1}{2}\exp(\gamma_{ij} x_{ij}^{\sigma\sigma'} (n_{i\sigma} - n_{j\sigma'}))\right],\\  
 &=\Pi_{ij}\sum_{x_{ij}^{\sigma\sigma'}=\pm 1} \frac{1}{2} \exp\left[\left(\gamma_{ij} x_{ij}^{\sigma\sigma'}-\frac{\Delta\tau U_{ij}}{2}\right) n_{i\sigma} - \left(\gamma_{ij} x_{ij}^{\sigma\sigma'}+\frac{\Delta\tau U_{ij}}{2}\right) n_{j\sigma'}\right],\\
&=\Pi_{ij}\sum_{x_{ij}^{\sigma\sigma'}=\pm 1} \frac{1}{2} \hat{B}_{ij}^{\sigma\sigma'}(x_{ij}^{\sigma\sigma'})
\end{split}
\end{equation*}
where $\cosh(\gamma_{ij}) = \exp \left(\frac{\Delta\tau U_{ij}}{2}\right)$. We will drop the indices on the $\hat{B}$ operators for simplicity. We also define

\begin{equation*}
\lambda_{\pm}(x_{ij}) = \pm \left(\gamma_{ij} x_{ij}^{\sigma\sigma'}\mp\frac{\Delta\tau U_{ij}}{2}\right).
\end{equation*}

Using this notation, we have

\begin{equation*}
\hat{B} = e^{\lambda_+n_{i\sigma}}e^{\lambda_-n_{j\sigma'}} = 1 + (e^{\lambda_+}-1)n_{i\sigma} + (e^{\lambda_-}-1)n_{j\sigma'} + (e^{\lambda_+}-1)(e^{\lambda_-}-1)n_{i\sigma}n_{j\sigma'},
\end{equation*}
where the $x_{ij}$ dependence has been suppressed.


The action of the exponential operator on a Slater determinant takes a particularly simple form in the site basis. Let's introduce the notation Diag $(\{i_1, d_{i_1}\}, \{i_2, d_{i_2}\},\dots)$ to denote the diagonal matrix with the elements $d_{i_1}, d_{i_2},\dots$ on the diagonal at indices $i_1, i_2,\dots$ and one at all other indices. Merging spin and orbital indices for the moment, the action of the exponential operator on a Slater determinant $\ket{\phi}$ leads to another Slater determinant $\ket{\phi'}$ with its matrix given by
\begin{equation*}
\ket{\phi'} = \hat{B}\ket{\phi} \rightarrow \phi' = B\phi = \text{Diag}(\{i,\exp(\lambda_{+})\}, \{j,\exp(\lambda_{-})\}) \phi.
\end{equation*}


The auxiliary fields can be trivially sampled from the uniform distribution, but using importance sampling one can reduce the noise significantly. This is accomplished by choosing $x_{ij}$ according to

\begin{equation*}
\tilde{p}(x_{ij}) \propto \text{max}\left(0, \frac{\bra{\psi_T}\phi'\rangle}{\bra{\psi_T}\phi\rangle}\right),
\end{equation*}
where $\ket{\psi_T}$ is a trial state. In addition to importance sampling this also enforces the positivity constraint. It guides the random walk towards regions where overlap with the trial state is large. When the trial state is a HF state, the overlap is given by
\begin{equation*}
\bra{\psi_T}\phi\rangle = \det(\psi_{T}^{\dagger} \phi) = \det(O(\phi)),
\end{equation*}
assuming a GHF trial state. For $n_e = n_{e\uparrow} + n_{e\downarrow}$ electrons in $n_o$ orbitals, $\psi_T$ is a $2n_0\times n_e$ matrix which we can think of being made up of two blocks. $O(\phi)$ is a $n_e \times n_e$ matrix, called the overlap matrix.


In [4]:
import numpy as np
import scipy as sp

np.set_printoptions(precision=4, suppress=True)
np.random.seed(0)

n_e = (4, 3)
n_o = 6
psi_t_up = np.random.randn(n_o, sum(n_e))
psi_t_down = np.random.randn(n_o, sum(n_e))
psi_t = np.vstack((psi_t_up, psi_t_down))
phi_up = np.random.randn(n_o, n_e[0])
phi_down = np.random.randn(n_o, n_e[1])
phi = sp.linalg.block_diag(phi_up, phi_down)
overlap_mat = psi_t.T.conj() @ phi
overlap = np.linalg.det(overlap_mat)

With a UHF walker and a GHF trial state, we can often save computational time by using the block structure.

\begin{equation*}
O(\phi) = \begin{pmatrix}
\psi_{T\uparrow}^{\dagger} & \psi_{T\downarrow}^{\dagger}\\
\end{pmatrix} \begin{pmatrix}
\phi_{\uparrow} & 0\\
0 & \phi_{\downarrow}\\  
 \end{pmatrix} = \begin{pmatrix}
\psi_{T\uparrow}^{\dagger}\phi_{\uparrow} & \psi_{T\downarrow}^{\dagger}\phi_{\downarrow}\\
\end{pmatrix}.
\end{equation*}


In [5]:
overlap_mat_up = psi_t_up.T.conj() @ phi_up
overlap_mat_down = psi_t_down.T.conj() @ phi_down
np.allclose(overlap_mat, np.hstack((overlap_mat_up, overlap_mat_down)))

True

But while deriving the fast update expressions, we will assume a GHF structure for the walkers as well, and suppress the spin indices.


Importance sampling requires the calculation of overlap ratios of Slater determinants given by

\begin{equation*}
\frac{O(\phi')}{O(\phi)} = \frac{\bra{\psi_T}\hat{B}\ket{\phi}}{\bra{\psi_T}\phi\rangle}.
\end{equation*}

The $\hat{B}$ operator contains up to two body terms, and therefore this ratio can be evaluated efficiently using Wick's theorem. The Green's function is given by

\begin{equation*}
G(\phi)_{pq} = \frac{\bra{\psi_T}c_p^{\dagger}c_q\ket{\phi}}{\bra{\psi_T}\phi\rangle} = (\phi O(\phi)^{-1} \psi_T^{\dagger})_{qp}.
\end{equation*}

Looking at the expression for $B$, we need the following elements:

\begin{equation*}
\begin{split}
\frac{\bra{\psi_T}n_{i}\ket{\phi}}{\bra{\psi_T}\phi\rangle} &= G(\phi)_{ii},\\
\frac{\bra{\psi_T}n_{j}\ket{\phi}}{\bra{\psi_T}\phi\rangle} &= G(\phi)_{jj},\\
\frac{\bra{\psi_T}n_{i}n_{j}\ket{\phi}}{\bra{\psi_T}\phi\rangle} &= G(\phi)_{ii}G(\phi)_{jj} - G(\phi)_{ij}G(\phi)_{ji}.
\end{split}
\end{equation*}
In the final line, we have used the fact that $i\neq j$.

Using these we get for the overlap ratio

\begin{equation*}
\frac{\bra{\psi_T}\hat{B}\ket{\phi}}{\bra{\psi_T}\phi\rangle} = (1 + (e^{\lambda_+}-1)G(\phi)_{ii})(1 + (e^{\lambda_-}-1)G(\phi)_{jj}) - (e^{\lambda_+}-1)(e^{\lambda_-}-1)G(\phi)_{ij}G(\phi)_{ji}.
\end{equation*}


In [6]:
# spin indices convention: all up followed by all down
i = 0
j = 3
lambda_p = np.random.randn()
lambda_n = np.random.randn()
cons_p = np.exp(lambda_p) - 1
cons_n = np.exp(lambda_n) - 1
phi_p = phi.copy()
phi_p[i] = np.exp(lambda_p) * phi[i]
phi_p[j] = np.exp(lambda_n) * phi[j]
overlap_mat_p = psi_t.T.conj() @ phi_p
overlap_p = np.linalg.det(overlap_mat_p)
ratio = overlap_p / overlap
green = (phi @ np.linalg.inv(overlap_mat) @ psi_t.T.conj()).T
wick_ratio = (1 + cons_p * green[i, i]) * (
    1 + cons_n * green[j, j]
) - cons_p * cons_n * green[i, j] * green[j, i]
np.allclose(ratio, wick_ratio)

True

While this saves a costly $O(N^3)$ determinant calculation, it requires the use of the Green's function which itself requires $O(N^3)$ effort. But there is a way to update the Green's function efficiently in $O(N^2)$ time. To see this we use Wick's theorem again:

\begin{equation*}
G(\phi')_{pq} = \frac{\bra{\psi_T}c_p^{\dagger}c_q\hat{B}\ket{\phi}}{\bra{\psi_T}\hat{B}\ket{\phi}} = \frac{\bra{\psi_T}c_p^{\dagger}c_q\hat{B}\ket{\phi}}{\bra{\psi_T}\phi\rangle} / \frac{\bra{\psi_T}\hat{B}\ket{\phi}}{\bra{\psi_T}\phi\rangle}.
\end{equation*}

We recognize the denominator as the overlap ratio we just calculated. Looking at the expression for $\hat{B}$, we see that the numerator consists of the following terms:

\begin{equation*}
\begin{split}
\frac{\bra{\psi_T}c_p^{\dagger}c_qn_i\ket{\phi}}{\bra{\psi_T}\phi\rangle} &= G(\phi)_{pq}G(\phi)_{ii} - G(\phi)_{pi}(G(\phi)-I)_{iq},\\
\frac{\bra{\psi_T}c_p^{\dagger}c_qn_j\ket{\phi}}{\bra{\psi_T}\phi\rangle} &= G(\phi)_{pq}G(\phi)_{jj} - G(\phi)_{pj}(G(\phi)-I)_{jq},\\
\frac{\bra{\psi_T}c_p^{\dagger}c_qn_in_j\ket{\phi}}{\bra{\psi_T}\phi\rangle} &= \det G'(\phi)_{pij,qij}.
\end{split}
\end{equation*}

In [7]:
green_p = (phi_p @ np.linalg.inv(overlap_mat_p) @ psi_t.T.conj()).T
# using wicks theorem
g_i = green * green[i, i] - np.outer(green[:, i], green[i])
g_j = green * green[j, j] - np.outer(green[:, j], green[j])
g_ij_1 = green * (green[i, i] * green[j, j] - green[i, j] * green[j, i])
v_i = np.zeros(2 * n_o)
v_i[i] = 1.0
v_j = np.zeros(2 * n_o)
v_j[j] = 1.0
g_ij_2 = -np.outer(
    green[:, i], (green[j, j] * (green[i] - v_i) - green[i, j] * (green[j] - v_j))
)
g_ij_3 = np.outer(
    green[:, j], (green[j, i] * (green[i] - v_i) - green[i, i] * (green[j] - v_j))
)